In [ ]:
!wget https://storage.googleapis.com/openimages/v7/oidv7-train-annotations-human-imagelabels.csv

In [ ]:
import pandas as pd
df = pd.read_csv('oidv7-train-annotations-human-imagelabels.csv')

In [ ]:
!wget https://storage.googleapis.com/openimages/v7/oidv7-class-descriptions.csv

In [ ]:
tmp = pd.read_csv('oidv7-class-descriptions.csv')
code_to_classname = dict(zip(tmp.LabelName, tmp.DisplayName))
code_to_classname

In [ ]:
counts = df.LabelName.value_counts()
counts = {code_to_classname[x]: y for x, y in dict(counts).items()}
pd.DataFrame({'class': counts.keys(), 'count': counts.values()}) \
    .to_csv('human_verified_image_labels_class_counts.csv')

In [ ]:
!wget https://storage.googleapis.com/openimages/v7/oidv7-train-annotations-machine-imagelabels.csv

In [ ]:
import pandas as pd
df = pd.read_csv('oidv7-train-annotations-machine-imagelabels.csv', usecols=['LabelName'])

In [ ]:
counts = df.LabelName.value_counts()
counts = {code_to_classname[x]: y for x, y in dict(counts).items()}
pd.DataFrame({'class': counts.keys(), 'count': counts.values()}) \
    .to_csv('machine_generated_image_labels_class_counts.csv')

In [ ]:
!wget https://storage.googleapis.com/openimages/v6/oidv6-train-annotations-bbox.csv

In [ ]:
import pandas as pd
df = pd.read_csv('oidv6-train-annotations-bbox.csv', usecols=['LabelName'])

In [ ]:
counts = df.LabelName.value_counts()
counts = {code_to_classname[x]: y for x, y in dict(counts).items()}
pd.DataFrame({'class': counts.keys(), 'count': counts.values()}) \
    .to_csv('box_annotations_class_counts.csv')

In [ ]:
!wget https://storage.googleapis.com/openimages/v7/oidv7-train-annotations-point-labels.csv

In [ ]:
import pandas as pd
df = pd.read_csv('oidv7-train-annotations-point-labels.csv', usecols=['Label'])

In [ ]:
counts = df.Label.value_counts()
counts = {code_to_classname[x]: y for x, y in dict(counts).items()}
pd.DataFrame({'class': counts.keys(), 'count': counts.values()}) \
    .to_csv('point_annotations_class_counts.csv')

In [ ]:
for i in range(10):
  url = f'https://storage.googleapis.com/localized-narratives/annotations/open_images_train_v6_localized_narratives-{i:05d}-of-00010.jsonl'
  !wget {url}

In [ ]:
captions = []
from tqdm import tqdm
import json

for i in range(10):
  filename = f'open_images_train_v6_localized_narratives-{i:05d}-of-00010.jsonl'
  with open(filename) as file:
    for line in tqdm(file):
      captions.append(json.loads(line.rstrip())['caption'])

In [ ]:
import pandas as pd
pd.DataFrame({'Text': pd.Series(captions)}).to_csv('narratives.csv')

In [ ]:
# now posting narratives to Solr search engine

In [15]:
!cp -r ../../xmls/simple_text_search /var/solr/data/open_images_narratives

In [16]:
!curl "http://localhost:8983/solr/admin/cores?action=CREATE&name=open_images_narratives"

{
        "responseHeader":{
          "status":0,
          "QTime":1475
        },
        "core":"open_images_narratives"
      }

In [17]:
%%bash
/opt/solr/bin/post -c open_images_narratives narratives.csv

java -classpath /opt/solr/server/solr-webapp/webapp/WEB-INF/lib/solr-core-9.3.0.jar -Dauto=yes -Dc=open_images_narratives -Ddata=files org.apache.solr.cli.SimplePostTool narratives.csv
SimplePostTool version 5.0.0
Posting files to [base] url http://localhost:8983/solr/open_images_narratives/update...
Entering auto mode. File endings considered are xml,json,jsonl,csv,pdf,doc,docx,ppt,pptx,xls,xlsx,odt,odp,ods,ott,otp,ots,rtf,htm,html,txt,log
POSTing file narratives.csv (text/csv) to [base]
1 files indexed.
COMMITting Solr index changes to http://localhost:8983/solr/open_images_narratives/update...
Time spent: 0:00:34.641


In [19]:
%%bash
curl "http://localhost:8983/solr/open_images_narratives/select?defType=edismax&df=Text&q=badger&start=0&rows=10"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1256  100  1256    0     0   2089      0 --:--:-- --:--:-- --:--:--  2089


{
        "responseHeader":{
          "status":0,
          "QTime":301,
          "params":{
            "q":"badger",
            "defType":"edismax",
            "df":"Text",
            "start":"0",
            "rows":"10"
          }
        },
        "response":{
          "numFound":3,
          "start":0,
          "numFoundExact":true,
          "docs":[{
            "Text":["In this picture we can see badger on the grass."],
            "id":"660ec566-4a11-4ee2-8222-2b6b00840a2e",
            "Unnamed__0":[344856],
            "_version_":1775460401492787207
          },{
            "Text":["In this image we can see a badger on the land. On the backside we can see a wall."],
            "id":"c6a6b033-9751-4acf-b5d5-f244541652d5",
            "Unnamed__0":[32722],
            "_version_":1775460384967229447
          },{
            "Text":["In this image I can see the toys which are in black, red, blue and yellow color. These are on the ash color surface. In the backgroun